---
# 회원 위치 정보 표시 어플을 만들자!
---
# 1. 데이터 정리하기
## 1.1 패키지 설치 및 불러오기

* 패키지 설치

```python
# 주소 - 위경도 변환 패키지
!pip install geopy

# 지도 시각화 패키지
!pip install folium

# application 제작 툴
!pip install streamlit
!pip install streamlit_folium

# protobuf 버전 확인
!pip list | grep protobuf
!pip install protobuf==3.20.1

```

###
### 패키지 불러오기

In [1]:
import pandas as pd
import numpy as np

from geopy.geocoders import Nominatim
import folium

In [2]:
# !pip list | grep proto

In [3]:
import streamlit

## 1.2 데이터 컬럼 추출
* .xlsx 확장자 -> .csv?

In [4]:
file_name = './data/information.xlsx'
df = pd.read_excel(file_name)

In [5]:
# df.head()

\
사용할 데이터를 추리기 위해 컬럼 명을 확인하자\
지도에 표시할 주소를 불러오고,\
이름 전공 등 구분가능한 간단한 정보들도 함께 정리

In [6]:
print(df.keys())
len(df.keys())

Index([' ', '분류/wave', '이름', '커뮤', '생년월일', '휴대폰번호', '이메일', '주소', '인스타 / 카카오계정',
       '거주국가', '지역', '웰컴미팅여부', 'Basic수료여부', '직업/전공', '목적', '가입일자', 'Slack이름',
       'Slack 계정주소', 'Slack활성(Y/N)'],
      dtype='object')


19

In [7]:
df.shape

(20, 19)

In [8]:
df = pd.DataFrame(df, columns=['이름', '커뮤', '주소', '직업/전공'])

# 시별 분류
df['city'] = df['주소'].str.split().str[0]

# 지역구별 분류
df['district'] = df['주소'].str.split().str[1]

# 상세주소 제거
df['address_geo'] = df['city'] + ' ' + df['district']
# df['address'] = df['주소'].str.split().str[0:2]

df = df.drop(['주소', 'city', 'district'], axis=1)

In [9]:
df['이름'] = df['이름'].str[0]

In [10]:
df.head()

,이름,커뮤,직업/전공,address_geo
0,강,Z,NaN,서울특별시 종로구
1,강,X,NaN,세종특별자치시 다정북로
2,강,X,NaN,부산 서구
3,강,X,NaN,부산 영도구
4,강,X,과외선생님/ 디자인,서울 종로구


## 1.3 주소 데이터로 위경도 변환하기
* 주소 컬럼에서 지역구 단위까지만 추출

In [11]:
geo_local = Nominatim(user_agent='South Korea')

# 주소를 위경도로 변환해주는 함수
def geocoding(address: pd.DataFrame) -> [float, float]:
    geo = geo_local.geocode(address)
    x_y = [geo.latitude, geo.longitude]
    return x_y

* 추출된 위도, 경도를 컬럼으로 저장

In [12]:
geo_local.geocode('서울시')

Location(서울, 대한민국, (37.5666791, 126.9782914, 0.0))

In [13]:
latitude, longitude = [], []

for idx, add in enumerate(df['address_geo']):
    # print(idx, add)
    latitude.append(geocoding(add)[0])
    longitude.append(geocoding(add)[1])
    
df['latitude'] = latitude
df['longitude'] = longitude
# df.head()

# 
---
# 2. 좌표를 지도로 표시하기
* 지도에 표시할 초기 위치를 지정한다

### `기본 지도` 
    -> 서울 중심을 기준으로 표시해볼까?

In [14]:
# map instanace init
m = folium.Map(location=[36, 128],   # 초기 위치 지정
               zoom_start=7,
               # width=750,
               # height=500
              )
m

### `마커 추가`
* `popup` : 표기할 팝업 문구 지정(마우스 클릭 시 표기되는 문구)
* `tooltip` : 표기할 툴팁 지정(마우스 오버 시 표기되는 문구)

In [15]:
# ```python
def add_marker(df):
    for idx, _ in enumerate(df.values):
        latitude_1 = df['latitude'][idx]
        longitude_1 = df['longitude'][idx]
        name_1 = df['이름'][idx]
        job_1 = df['직업/전공'][idx]
        comm_1 = df['커뮤'][idx]
        popup_1 = folium.Popup(f'{name_1} : {job_1}',
                               max_width=80)
        tooltip_1 = df['이름'][idx]

        # 커뮤별 아이콘 색 분류
        if comm_1 == 'Z':
            color = "blue"
        elif comm_1 == 'M':
            color = "green"
        else:
            color = "red"

        # folium.Marker([latitude_1, longitude_1],
        #               popup=popup_1,
        #               tooltip=tooltip_1
        #              ).add_to(m)
        
        folium.Circle([latitude_1, longitude_1],
                      popup=popup_1,
                      tooltip=tooltip_1,
                      icon = folium.Icon(color=color),
                      radius=200
                     ).add_to(m)
        
add_marker(df)
m
# ```
### 

### `마커 클러스터 적용하기`
* 인접 지역에 해당하는 데이터를 인터렉티브하게 표시해주는 기능
* 행정구역 표시까지 추가하면 좋을 듯 -> Ref. 참고

In [16]:
from folium.plugins import MarkerCluster

In [17]:
def add_cluster(df):
    # 클러스터 적용
    marker_cluster = MarkerCluster().add_to(m)
    
    for idx, _ in enumerate(df.values):
        # 위경도
        lat = df['latitude'][idx]
        long = df['longitude'][idx]
        
        # 추가 정보
        name_1 = df['이름'][idx]
        job_1 = df['직업/전공'][idx]
        comm_1 = df['커뮤'][idx]
        popup_1 = folium.Popup(f'{name_1} : {job_1}',
                               max_width=80)
        tooltip_1 = f'{name_1} : {comm_1}'
        
        # 커뮤별 아이콘 색 분류
        if comm_1 == 'Z':
            color = "blue"
        elif comm_1 == 'M':
            color = "green"
        else:
            color = "red"
        
        # 마커 추가
        folium.Marker([lat, long], 
                      icon = folium.Icon(color=color),   # color='darkblue' -> 아보트블루
                      popup=popup_1,
                      tooltip=tooltip_1,
                     ).add_to(marker_cluster)

add_cluster(df)
m

In [18]:
# html 파일 저장
# m.save('map.html')

# 3. Streamlit
## 3.1 실행파일 만들기 - test

* 파일 실행 - terminal
```
$ steamlit run st_fol_test.py
```


In [19]:
%%writefile st_fol_test.py
# 라이브러리 로드
import streamlit as st
from streamlit_folium import st_folium
import folium
from folium.plugins import MarkerCluster

import pandas as pd
import numpy as np

# 위경도 변환
from geopy.geocoders import Nominatim

Overwriting st_fol_test.py


In [20]:
%%writefile -a st_fol_test.py
# ===============================================================
# Functions
# 주소를 위경도로 변환해주는 함수
def geocoding(address: pd.Series) -> [float, float]:
    geo = geo_local.geocode(address)
    x_y = [geo.latitude, geo.longitude]
    return x_y

# 지도에 마커 추가
def add_marker(df):
    for idx, _ in enumerate(df.values):
        lat = df['latitude'][idx]
        long = df['longitude'][idx]
        
        name_1 = df['이름'][idx]
        job_1 = df['직업/전공'][idx]
        comm_1 = df['커뮤'][idx]
        
        popup_1 = folium.Popup(f'{name_1} : {job_1}',
                               max_width=80)
        tooltip_1 = f'{name_1} : {comm_1}'

        
        # 커뮤별 아이콘 색 분류, circle 적용 불가
        if comm_1 == 'Z':
            color = "blue"
        elif comm_1 == 'M':
            color = "green"
        else:
            color = "red"
            
        # folium.Marker([latitude_1, longitude_1],
        #               popup=popup_1,
        #               tooltip=tooltip_1
        #              ).add_to(m)
        
        folium.Circle([lat, long],
                      popup=popup_1,
                      tooltip=tooltip_1,
                      radius=300
                     ).add_to(st.session_state.fol_map)

# 클러스터 마커 추가        
def add_cluster(df):
    # 클러스터 적용
    marker_cluster = MarkerCluster().add_to(st.session_state.fol_map)
    
    for idx, _ in enumerate(df.values):
        # 위경도
        lat = df['latitude'][idx]
        long = df['longitude'][idx]
        
        # 추가 정보
        name_1 = df['이름'][idx]
        job_1 = df['직업/전공'][idx]
        comm_1 = df['커뮤'][idx]
        popup_1 = folium.Popup(f'{name_1} : {job_1}',
                               max_width=80)
        tooltip_1 = f'{name_1} : {comm_1}'
        
        # 커뮤별 아이콘 색 분류
        if comm_1 == 'Z':
            color = "blue"
        elif comm_1 == 'M':
            color = "green"
        else:
            color = "red"
        
        # 마커 추가
        folium.Marker([lat, long], 
                      icon = folium.Icon(color=color),   # color='darkblue' -> 아보트블루
                      popup=popup_1,
                      tooltip=tooltip_1,
                     ).add_to(marker_cluster)

Appending to st_fol_test.py


In [21]:
%%writefile -a st_fol_test.py
        
# ===============================================================
# Streamlit Session State Initialization
if 'fol_map' not in st.session_state:
    st.session_state.fol_map = []
    

# ===============================================================
# Main loop

# Main page title
st.set_page_config(page_title="Streamlit + Folium 으로 회원정보 지도 만들기")
st.title('📸 Streamlit + Folium 으로 회원정보 지도 만들기')

# Sidebar
page = st.sidebar.radio(
    "Select page", 
    ["File upload", "Circle marker", "Cluster marker"], 
    index=0,
)


Appending to st_fol_test.py


In [22]:
%%writefile -a st_fol_test.py

# Page 1 File upload
if page == 'File upload':
# file upload button - csv, xlsx 파일형식

    st.header('File upload')
    uploaded_file = st.file_uploader("Choose a file - .csv or .xlsx only", type=['csv', 'xlsx'])
    
    if uploaded_file is not None:
        with st.spinner('Processing! Wait for it...'):
            # 데이터프레임 전처리
            df = pd.read_excel(uploaded_file)
            df = pd.DataFrame(df, columns=['이름', '커뮤', '주소', '직업/전공'])

            df['city'] = df['주소'].str.split().str[0]   # 시별 분류
            df['district'] = df['주소'].str.split().str[1]   # 지역구별 분류
            df['address_geo'] = df['city'] + ' ' + df['district']   # 상세 주소 
            st.session_state.df = df.drop(['city', 'district'], axis=1)   # 불필요한 컬럼 제거

            # 상세 주소로 위경도 변환
            geo_local = Nominatim(user_agent='South Korea')
            latitude, longitude = [], []

            for idx, add in enumerate(st.session_state.df['address_geo']):
                # print(idx, add)
                latitude.append(geocoding(add)[0])
                longitude.append(geocoding(add)[1])

            st.session_state.df['latitude'] = latitude
            st.session_state.df['longitude'] = longitude

        st.success('Done!')   # Spinner 완료
        st.write(st.session_state.df)   # plot dataframe

Appending to st_fol_test.py


In [23]:
%%writefile -a st_fol_test.py

# Page 2 Circle marker
elif page == 'Circle marker':
    st.header('Circle marker map')
    
    # reset map instance
    st.session_state.fol_map = folium.Map(location=[36, 128],   # 초기 위치 지정
               zoom_start=7,
               # width=750,
               # height=500
              )
    
    add_marker(st.session_state.df)
    # call to render Folium map in Streamlit
    with st.spinner('Wait for it...'):
        st_data = st_folium(st.session_state.fol_map, width = 725)
        st.write(st.session_state.df)
    st.success('Done!')
    

Appending to st_fol_test.py


In [24]:
%%writefile -a st_fol_test.py

# Page 3 Cluster marker
elif page == 'Cluster marker':
    st.header('Cluster marker map')
    
    # reset map instance
    st.session_state.fol_map = folium.Map(location=[36, 128],   # 초기 위치 지정
               zoom_start=7,
               # width=750,
               # height=500
              )
    
    add_cluster(st.session_state.df)    
    # call to render Folium map in Streamlit
    with st.spinner('Wait for it...'):
        st_data = st_folium(st.session_state.fol_map, width = 725)
        st.write(st.session_state.df)
    st.success('Done!')

Appending to st_fol_test.py


In [25]:
# .py 파일내용 불러오기
%less st_fol_test.py

# 라이브러리 로드
import streamlit as st
from streamlit_folium import st_folium
import folium
from folium.plugins import MarkerCluster

import pandas as pd
import numpy as np

# 위경도 변환
from geopy.geocoders import Nominatim
# ===============================================================
# Functions
# 주소를 위경도로 변환해주는 함수
def geocoding(address: pd.Series) -> [float, float]:
    geo = geo_local.geocode(address)
    x_y = [geo.latitude, geo.longitude]
    return x_y

# 지도에 마커 추가
def add_marker(df):
    for idx, _ in enumerate(df.values):
        lat = df['latitude'][idx]
        long = df['longitude'][idx]
        
        name_1 = df['이름'][idx]
        job_1 = df['직업/전공'][idx]
        comm_1 = df['커뮤'][idx]
        
        popup_1 = folium.Popup(f'{name_1} : {job_1}',
                               max_width=80)
        tooltip_1 = f'{name_1} : {comm_1}'

        
        # 커뮤별 아이콘 색 분류, circle 적용 불가
        if comm_1 == 'Z':
            color = "blue"
        elif comm_1 == 'M':
            col

#
---
# 4. Streamlit Web App. 배포하기
---

-> 디렉토리 구조화 -> Github 업로드

Option 1 :
[Streamlit Sharing](https://lucaseo.github.io/posts/2021-10-09-intro-streamlit-sharing/)

Option 2 :
[Heroku app. 으로 배포하기](https://lucaseo.github.io/posts/2020-03-29-deploy-streamlit-to-heroku/)

---
# Reference

[지도 시각화 도구 Folium 사용법을 파헤쳐보자!](https://teddylee777.github.io/visualization/folium)\
[[Python] 전국 인구 현황 지도 시각화](https://mkjjo.github.io/python/2019/08/18/korea_population.html)


[Streamlit-Folium](https://github.com/randyzwitch/streamlit-folium/blob/master/examples/streamlit_app.py)\
[Streamlit Map Example](https://docs.streamlit.io/library/api-reference/charts/st.map)\
[Streamlit App Page Example](https://github.com/randyzwitch/streamlit-folium/blob/master/examples/streamlit_app.py)


---